In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise03AI import Exercise03AI
from Exercise02AI import display_path

# Aufgabe 04: Minimax (Simplified Evaluation Function) mit Memoisierung

Diese Klasse implementiert die Memoisierungsfunktion für den Minimax-Algorithmus zur Berechnung des nächsten Zuges im Mittelspiel. Hierbei wird die `minimax`-Funktion der Super-Klasse (`Exercise03AI`) aufgerufen und durch die nun implementierte Memoisierung beschleunigt.

In [ ]:
import chess

class Exercise04AI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm and piece square tables"""
    def __init__(self, player_name: str, search_depth: int = 3) -> None:
        super().__init__(player_name, search_depth)

## Memoisierungsfunktion

Die Klasse wird um eine Variable `cache` erweitert. Initial ist `cache` ein leeres Dictionary, welches als Schlüssel einen String annimmt und als Wert ein beliebiges Objekt zurückgibt.

Die Memoisierungsfunktion `memoize` nimmt eine Funktion `f` als Argument und gibt eine memoisierte Version `memoized_f` dieser Funktion zurück.

Die memoisierte Funktion `memoized_f` versucht zunächst, den Rückgabewert der Ursprungsfunktion aus dem `cache` auszulesen. Dabei werden als Schlüssel die gegebenen Funktionsargumente `args` als String verwendet. Beinhaltet der `cache` ein Ergebnis für die gegebenen Argumente `args`, so wird dieses zurückgegeben. Andernfalls wird die Ursprungsfunktion `f` aufgerufen, der Rückgabewert wird für die verwendeten Argumente im `cache` hinterlegt und schlussendlich wird der berechnete Wert zurückgegeben.

### Cache-Schlüssel

Eine wichtige Rolle spielt der Key, welcher für die Speicherung bereits berechneter Ergebnisse im `cache`-Dictionary verwendet wird. Um den Cache möglichst effektiv nutzen zu können, sollten nur die Attribute in die Berechnung des Schlüssels miteinbezogen werden, welche auch durch den Minimax-Algorithmus verwendet werden und sich im Laufe des Spieles verändern. Aus diesem Grund ist beispielsweise die eigene Klasseninstanz (`self`-Argument) kein Bestandteil des Schlüssels. Das Board wird mithilfe der FEN-Notation im Schlüssel inkludiert und mit den restlichen Argumenten als String konkateniert. Dabei wird der Zähler für die Nummer des nächsten Zuges (letzte Stelle) von der FEN-Notation des Boards entfernt, da er nicht für die Minimax Berechnung erforderlich ist und die Wiederverwertbarkeit von Berechnungen stark einschränkt. Der Zähler für die Halbzüge seit dem letzten Bauernzug oder Schlagen einer Figur (vorletzte Stelle der FEN) wird genau dann beibehalten, wenn er für das Erkennen der `fifty-move`-Regel (Remis) relevant ist, das heißt wenn $$\texttt{Zähler} \; \ge \; 50 - \texttt{Tiefe}$$ gilt, wobei $\texttt{Tiefe}$ die Suchtiefe des aktuellen Aufrufs der Minimax-Funktion ist. Da auch die `fivefold-repetition`-Regel (fünfmalige Stellungswiederholung) für die Berechnung relevant ist, wird die Anzahl der aktuellen Wiederholungen ebenfalls in den Cache-Schlüssel mit einbezogen.

Die Funktion `memoize` wird als _Decorator_ auf die `minimax` Funktion angewendet.

In [ ]:
from typing import Any

class Exercise04AI(Exercise04AI): # type: ignore
    cache: dict[tuple, Any] = {}
    cache_hits = 0

    @staticmethod
    def memoize(f):
        def f_memoized(*args):
            fen_list = args[1].fen().split(" ")
            if int(fen_list[-2]) >= (50 - args[2]):
                # Include fifty moves counter if it has a relevant value
                reduced_fen = " ".join(fen_list[:-1])
            else:
                # Skip fifty moves counter otherwise
                reduced_fen = " ".join(fen_list[:-2])
            key = reduced_fen + str(args[2]) + str(args[0].maybe_repetitions(args[1]))
            if key in Exercise04AI.cache:
                Exercise04AI.cache_hits += 1
                return Exercise04AI.cache[key]
            result = f(*args)
            Exercise04AI.cache[key] = result
            return result

        return f_memoized
    
    @memoize
    def minimax(self, *args) -> int:
        """Memoized version of the Exercise03AI minimax implementation"""
        return super().minimax(*args)

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
import chess.pgn
from os.path import join

with open(join("..", "games", "2022-01-15_20-18-57-907777.pgn")) as pgn:
    first_game = chess.pgn.read_game(pgn)

# Iterate through all moves and play them on a board.
board = first_game.board()
for move in first_game.mainline_moves():
    board.push(move)
for i in range(220):
    board.pop()

In [ ]:
%%time
DEPTH = 2
#board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
player.cache_hits

In [ ]:
%%time
DEPTH = 3
board = chess.Board("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
display_path(board, move, DEPTH)

In [ ]:
board = chess.Board("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise04AI("Testplayer", 3)
print(player.evaluate(board))
board